In [ ]:
!ls /home/aistudio/data/data199606

In [4]:
!unzip /home/aistudio/data/data199606/wurenji.zip -d /home/aistudio/ >/dev/null

In [1]:
!pip install sahi   > /dev/null
!pip install pylabel > /dev/null
!pip install pycocotools > /dev/null

In [2]:
!nvidia-smi

Sun Mar 19 14:33:39 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:06:00.0 Off |                    0 |
| N/A   35C    P0    39W / 300W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import json
import os
root = '/home/aistudio/wurenji/train/'
dir = os.listdir(root)
from sahi.utils.coco import Coco, CocoCategory, CocoImage, CocoAnnotation
from sahi.utils.file import save_json
from PIL import Image

coco = Coco()
coco.add_category(CocoCategory(id=0, name='uav'))
for folder in dir:
  print(folder)
  with open(root+folder+'/IR_label.json','r',encoding='utf8') as fp1:
      json_data = json.load(fp1)
      num = len(os.listdir(root+folder))
      print(num)
      
      for i in range(1,num-1):
        if i%10!=3:
          os.remove(root+folder+"/"+str(i).zfill(6)+".jpg")
          continue
        imgname = root+folder+"/"+str(i).zfill(6)+".jpg"
        width, height = Image.open(imgname).size
        coco_image = CocoImage(file_name=imgname, height=height, width=width)
        ann = json_data["gt_rect"][i-1]
        if(json_data['exist'][i-1]==0):
          continue
        # print(root+folder+str(i))
        coco_image.add_annotation(
            
            CocoAnnotation(
                bbox=[ann[0], ann[1], ann[2], ann[3]],
                category_id=0,
                category_name='uav'
            )
        )
        coco.add_image(coco_image)
!mkdir ./wurenji/annotations
save_json(data=coco.json, save_path='./wurenji/annotations/test.json')
print("------------")

In [ ]:
# from pylabel import importer
# dataset = importer.ImportCoco("./test.json", path_to_images="./wurenji/train", name="uav")
# dataset.path_to_annotations = "data/yoloformat"
# dataset.export.ExportToYoloV5()[0]

In [ ]:
!git clone https://gitee.com/monkeycc/PaddleYOLO.git # clone
!cd PaddleYOLO
!pip install -r PaddleYOLO/requirements.txt  # install

## 训练

In [5]:
import pycocotools
!python -m paddle.distributed.launch --selected_gpus="0" ./PaddleYOLO/tools/train.py -c ./PaddleYOLO/configs/yolov7/yolov7_tiny_300e_coco.yml --eval --amp 

LAUNCH WARNING 2023-03-18 20:46:59,362 Compatible mode enable with args ['--selected_gpus=0']
-----------  Configuration Arguments -----------
backend: auto
cluster_topo_path: None
coordinator_num: None
coordinators: 
elastic_pre_hook: None
elastic_server: None
enable_auto_mapping: False
force: False
gpus: 0
heter_devices: 
heter_worker_num: None
heter_workers: 
host: None
http_port: None
ips: 127.0.0.1
job_id: None
log_dir: log
np: None
nproc_per_node: None
rank_mapping_path: None
run_mode: None
scale: 0
server_num: None
servers: 
training_script: ./PaddleYOLO/tools/train.py
training_script_args: ['-c', './PaddleYOLO/configs/yolov7/yolov7_tiny_300e_coco.yml', '--eval', '--amp']
worker_num: None
workers: 
------------------------------------------------
[2023-03-18 20:46:59,364] [ WARNING] launch.py:538 - Not found distinct arguments and compiled with cuda or xpu or npu or mlu. Default use collective mode
WARNING 2023-03-18 20:46:59,364 launch.py:538] Not found distinct arguments and c

In [17]:
with open("/home/aistudio/output/bbox.json",'r',encoding='utf8') as fp1:
    bbox_data = json.load(fp1)
    print(bbox_data[1000])

{'image_id': 122, 'category_id': 1, 'bbox': [34.85061264038086, 331.8454895019531, 111.25436782836914, 57.656494140625], 'score': 0.007921426557004452}


## 单个文件

In [15]:
!rm -rf ./output/*.jpg

In [6]:
import os
os.system("python /home/aistudio/PaddleYOLO/tools/infer.py -c /home/aistudio/PaddleYOLO/configs/yolov7/yolov7_tiny_300e_coco.yml -o weights=/home/aistudio/output/yolov7_tiny_300e_coco/model_final.pdparams --infer_img=/home/aistudio/wurenji/test/01_2192_0001-1500/000541.jpg --save_results ./test --draw_threshold=0.1")

W0320 16:10:20.398725  1095 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.2, Runtime API Version: 11.2
W0320 16:10:20.403421  1095 gpu_resources.cc:91] device: 0, cuDNN Version: 8.2.


[03/20 16:10:22] ppdet.utils.checkpoint INFO: Finish loading model weights: /home/aistudio/output/yolov7_tiny_300e_coco/model_final.pdparams
[03/20 16:10:22] ppdet.data.source.category WARNING: anno_file 'dataset/coco/annotations/instances_val2017.json' is None or not set or not exist, please recheck TrainDataset/EvalDataset/TestDataset.anno_path, otherwise the default categories will be used by metric_type.
[03/20 16:10:22] ppdet.data.source.category WARNING: metric_type: COCO, load default categories of COCO.
[03/20 16:10:22] ppdet.data.source.category WARNING: anno_file 'dataset/coco/annotations/instances_val2017.json' is None or not set or not exist, please recheck TrainDataset/EvalDataset/TestDataset.anno_path, otherwise the default categories will be used by metric_type.
[03/20 16:10:22] ppdet.data.source.category WARNING: metric_type: COCO, load default categories of COCO.
[03/20 16:10:22] ppdet.engine INFO: Test loader length is 1, test batch_size is 1.
[03/20 16:10:22] ppdet.e

100%|██████████| 1/1 [00:01<00:00,  1.41s/it]


[03/20 16:10:23] ppdet.metrics.metrics INFO: The bbox result is saved to bbox.json.
[03/20 16:10:23] ppdet.metrics.metrics INFO: The bbox result is saved to output/bbox.json and do not evaluate the mAP.
[03/20 16:10:23] ppdet.engine INFO: Detection bbox results save in output/000541.jpg


0

In [1]:
import os
for i in range(1,10):
    os.system("python /home/aistudio/PaddleYOLO/tools/infer.py -c /home/aistudio/PaddleYOLO/configs/yolov7/yolov7_tiny_300e_coco.yml -o weights=/home/aistudio/output/yolov7_tiny_300e_coco/model_final.pdparams --infer_img=/home/aistudio/wurenji/test/wg2022_ir_020_split_08/"+str(i).zfill(6)+".jpg --save_results ./test ")

W0319 20:38:16.383507  1138 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.2, Runtime API Version: 11.2
W0319 20:38:16.387972  1138 gpu_resources.cc:91] device: 0, cuDNN Version: 8.2.


[03/19 20:38:18] ppdet.utils.checkpoint INFO: Finish loading model weights: /home/aistudio/output/yolov7_tiny_300e_coco/model_final.pdparams
[03/19 20:38:18] ppdet.data.source.category WARNING: anno_file 'dataset/coco/annotations/instances_val2017.json' is None or not set or not exist, please recheck TrainDataset/EvalDataset/TestDataset.anno_path, otherwise the default categories will be used by metric_type.
[03/19 20:38:18] ppdet.data.source.category WARNING: metric_type: COCO, load default categories of COCO.
[03/19 20:38:18] ppdet.data.source.category WARNING: anno_file 'dataset/coco/annotations/instances_val2017.json' is None or not set or not exist, please recheck TrainDataset/EvalDataset/TestDataset.anno_path, otherwise the default categories will be used by metric_type.
[03/19 20:38:18] ppdet.data.source.category WARNING: metric_type: COCO, load default categories of COCO.
[03/19 20:38:18] ppdet.engine INFO: Test loader length is 1, test batch_size is 1.
[03/19 20:38:18] ppdet.e

100%|██████████| 1/1 [00:01<00:00,  1.26s/it]


[03/19 20:38:19] ppdet.metrics.metrics INFO: The bbox result is saved to bbox.json.
[03/19 20:38:19] ppdet.metrics.metrics INFO: The bbox result is saved to output/bbox.json and do not evaluate the mAP.
[03/19 20:38:19] ppdet.engine INFO: Detection bbox results save in output/000001.jpg


W0319 20:38:22.258324  1259 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.2, Runtime API Version: 11.2
W0319 20:38:22.262508  1259 gpu_resources.cc:91] device: 0, cuDNN Version: 8.2.


...

[03/19 20:39:01] ppdet.metrics.metrics INFO: The bbox result is saved to bbox.json.
[03/19 20:39:01] ppdet.metrics.metrics INFO: The bbox result is saved to output/bbox.json and do not evaluate the mAP.
[03/19 20:39:01] ppdet.engine INFO: Detection bbox results save in output/000008.jpg


W0319 20:39:03.883507  1670 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.2, Runtime API Version: 11.2
W0319 20:39:03.887714  1670 gpu_resources.cc:91] device: 0, cuDNN Version: 8.2.


[03/19 20:39:06] ppdet.utils.checkpoint INFO: Finish loading model weights: /home/aistudio/output/yolov7_tiny_300e_coco/model_final.pdparams
[03/19 20:39:06] ppdet.data.source.category WARNING: anno_file 'dataset/coco/annotations/instances_val2017.json' is None or not set or not exist, please recheck TrainDataset/EvalDataset/TestDataset.anno_path, otherwise the default categories will be used by metric_type.
[03/19 20:39:06] ppdet.data.source.category WARNING: metric_type: COCO, load default categories of COCO.
[03/19 20:39:06] ppdet.data.source.category WARNING: anno_file 'dataset/coco/annotations/instances_val2017.json' is None or not set or not exist, please recheck TrainDataset/EvalDataset/TestDataset.anno_path, otherwise the default categories will be used by metric_type.
[03/19 20:39:06] ppdet.data.source.category WARNING: metric_type: COCO, load default categories of COCO.
[03/19 20:39:06] ppdet.engine INFO: Test loader length is 1, test batch_size is 1.
[03/19 20:39:06] ppdet.e

100%|██████████| 1/1 [00:01<00:00,  1.30s/it]


[03/19 20:39:07] ppdet.metrics.metrics INFO: The bbox result is saved to bbox.json.
[03/19 20:39:07] ppdet.metrics.metrics INFO: The bbox result is saved to output/bbox.json and do not evaluate the mAP.
[03/19 20:39:07] ppdet.engine INFO: Detection bbox results save in output/000009.jpg


In [11]:
import os
os.system("rm -rf /home/aistudio/output/bbox.json")
os.system("rm -rf /home/aistudio/output/*.jpg")

0

## 全部文件夹

In [1]:
import os
import json
folders = os.listdir("/home/aistudio/wurenji/test")
for folder in folders:
    os.system("python /home/aistudio/PaddleYOLO/tools/infer.py -c /home/aistudio/PaddleYOLO/configs/yolov7/yolov7_tiny_300e_coco.yml -o weights=/home/aistudio/output/yolov7_tiny_300e_coco/model_final.pdparams --infer_dir=/home/aistudio/wurenji/test/"+folder+" --save_results ./test --visualize False")
    bbox = "/home/aistudio/output/bbox"
    bbox_data=[]
    with open("/home/aistudio/output/bbox.json",'r',encoding='utf8') as fp1:
        bbox_data = json.load(fp1)
    os.system("mv /home/aistudio/output/bbox.json /home/aistudio/ans/"+folder+".json")
    os.system("rm -rf /home/aistudio/output/bbox.json")
    os.system("rm -rf /home/aistudio/output/*.jpg")


W0319 21:44:45.362495   628 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 8.0, Driver API Version: 11.2, Runtime API Version: 11.2
W0319 21:44:45.365794   628 gpu_resources.cc:91] device: 0, cuDNN Version: 8.2.


[03/19 21:44:47] ppdet.utils.checkpoint INFO: Finish loading model weights: /home/aistudio/output/yolov7_tiny_300e_coco/model_final.pdparams
['/home/aistudio/wurenji/test/20190926_183941_1_9/000001.jpg', '/home/aistudio/wurenji/test/20190926_183941_1_9/000002.jpg', '/home/aistudio/wurenji/test/20190926_183941_1_9/000003.jpg', '/home/aistudio/wurenji/test/20190926_183941_1_9/000004.jpg', '/home/aistudio/wurenji/test/20190926_183941_1_9/000005.jpg', '/home/aistudio/wurenji/test/20190926_183941_1_9/000006.jpg', '/home/aistudio/wurenji/test/20190926_183941_1_9/000007.jpg', '/home/aistudio/wurenji/test/20190926_183941_1_9/000008.jpg', '/home/aistudio/wurenji/test/20190926_183941_1_9/000009.jpg', '/home/aistudio/wurenji/test/20190926_183941_1_9/000010.jpg', '/home/aistudio/wurenji/test/20190926_183941_1_9/000011.jpg', '/home/aistudio/wurenji/test/20190926_183941_1_9/000012.jpg', '/home/aistudio/wurenji/test/20190926_183941_1_9/000013.jpg', '/home/aistudio/wurenji/test/20190926_183941_1_9/000

100%|██████████| 163/163 [00:06<00:00, 26.07it/s]


[03/19 21:44:53] ppdet.metrics.metrics INFO: The bbox result is saved to bbox.json.
[03/19 21:44:53] ppdet.metrics.metrics INFO: The bbox result is saved to output/bbox.json and do not evaluate the mAP.


W0319 21:44:58.027827   743 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 8.0, Driver API Version: 11.2, Runtime API Version: 11.2
W0319 21:44:58.030959   743 gpu_resources.cc:91] device: 0, cuDNN Version: 8.2.


...

[03/19 22:09:40] ppdet.metrics.metrics INFO: The bbox result is saved to bbox.json.
[03/19 22:09:40] ppdet.metrics.metrics INFO: The bbox result is saved to output/bbox.json and do not evaluate the mAP.



W0319 22:09:46.082791  9800 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 8.0, Driver API Version: 11.2, Runtime API Version: 11.2
W0319 22:09:46.085978  9800 gpu_resources.cc:91] device: 0, cuDNN Version: 8.2.


[03/19 22:09:47] ppdet.utils.checkpoint INFO: Finish loading model weights: /home/aistudio/output/yolov7_tiny_300e_coco/model_final.pdparams
['/home/aistudio/wurenji/test/3700000000002_105103_8/000001.jpg', '/home/aistudio/wurenji/test/3700000000002_105103_8/000002.jpg', '/home/aistudio/wurenji/test/3700000000002_105103_8/000003.jpg', '/home/aistudio/wurenji/test/3700000000002_105103_8/000004.jpg', '/home/aistudio/wurenji/test/3700000000002_105103_8/000005.jpg', '/home/aistudio/wurenji/test/3700000000002_105103_8/000006.jpg', '/home/aistudio/wurenji/test/3700000000002_105103_8/000007.jpg', '/home/aistudio/wurenji/test/3700000000002_105103_8/000008.jpg', '/home/aistudio/wurenji/test/3700000000002_105103_8/000009.jpg', '/home/aistudio/wurenji/test/3700000000002_105103_8/000010.jpg', '/home/aistudio/wurenji/test/3700000000002_105103_8/000011.jpg', '/home/aistudio/wurenji/test/3700000000002_105103_8/000012.jpg', '/home/aistudio/wurenji/test/3700000000002_105103_8/000013.jpg', '/home/aistud

100%|██████████| 1500/1500 [00:29<00:00, 51.08it/s]

[03/19 22:10:17] ppdet.metrics.metrics INFO: The bbox result is saved to bbox.json.
[03/19 22:10:17] ppdet.metrics.metrics INFO: The bbox result is saved to output/bbox.json and do not evaluate the mAP.


## 单个文件夹

In [ ]:
import os
import json
folder = "20190925_101846_1_1"
os.system("python /home/aistudio/PaddleYOLO/tools/infer.py -c /home/aistudio/PaddleYOLO/configs/yolov7/yolov7_tiny_300e_coco.yml -o weights=/home/aistudio/output/yolov7_tiny_300e_coco/model_final.pdparams --infer_dir=/home/aistudio/wurenji/test/"+folder+" --save_results ./test")
bbox = "/home/aistudio/output/bbox"
bbox_data=[]
with open("/home/aistudio/output/bbox.json",'r',encoding='utf8') as fp1:
    bbox_data = json.load(fp1)


In [ ]:
import json
vis=[0 for i in range(2000)]
ans=[]
with open("/home/aistudio/output/bbox.json",'r',encoding='utf-8') as fp1:
  json_data = json.load(fp1)
  for i in json_data:
    if vis[i["image_id"]]==1:
      continue
    print(i)
    vis[i["image_id"]]=1
    temp = []
    if i["score"]<0.05:
      print(str(i["image_id"])+"+++"+str(i["score"]))
      ans.append(temp)
      continue
    else:
      ans.append(i["bbox"])
      print("-------------"+str(i["score"])+"+"+str(i["image_id"]))
print(ans)

## 处理ans

In [12]:
import json
files = os.listdir("./ans")

for name in files:
    name = name[0:-5]
    if name[0]=='.':
        continue
    vis = [0 for i in range(1,2000)]
    res = {"res":[]}
    cnt = 0
    with open("/home/aistudio/ans/"+name+".json",'r',encoding='utf-8') as fp1:
        json_data = json.load(fp1)
        print(json_data)
        for it in json_data:
            temp=[]
            if vis[it["image_id"]]:
                continue
            if it["score"]<0.005:
                res["res"].append(temp)
                #print(cnt,end=" ")
                print(cnt)
                cnt = cnt+1
                vis[it["image_id"]]=1
                print(it["score"])
                continue
            vis[it["image_id"]]=1
            res["res"].append(it["bbox"])
            #print(cnt,end=" ")
    #print(str(res))
    print(cnt)
    f = open("./res/"+name+".txt","w")
    f.write(str(res))
    f.close()

        
        
      

[{'image_id': 0, 'category_id': 1, 'bbox': [309.0836181640625, 277.27557373046875, 57.59417724609375, 42.7196044921875], 'score': 0.42117321491241455}, {'image_id': 0, 'category_id': 1, 'bbox': [311.56396484375, 285.35858154296875, 45.2847900390625, 35.61505126953125], 'score': 0.012416101060807705}, {'image_id': 0, 'category_id': 1, 'bbox': [315.49481201171875, 280.38201904296875, 44.28729248046875, 37.62384033203125], 'score': 0.00379589363001287}, {'image_id': 0, 'category_id': 1, 'bbox': [300.18072509765625, 276.8687438964844, 53.46685791015625, 41.00286865234375], 'score': 0.0031706946901977062}, {'image_id': 0, 'category_id': 1, 'bbox': [317.4538269042969, 275.5685119628906, 51.50921630859375, 37.7520751953125], 'score': 0.0025574229657649994}, {'image_id': 1, 'category_id': 1, 'bbox': [306.0187683105469, 277.45660400390625, 59.17254638671875, 43.8590087890625], 'score': 0.40381067991256714}, {'image_id': 1, 'category_id': 1, 'bbox': [309.7928466796875, 284.787109375, 46.92791748

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)




20
0.002471354091539979
21
0.002502904972061515
22
0.0012882823357358575
23
0.002737391972914338
24
0.002002701163291931
25
0.0014650247758254409
26
0.0029893869068473577
27
0.0029199731070548296
28
0.0031945055816322565
29
0.0030555634293705225
30
0.003583502722904086
31
0.002406741725280881
32
0.002227527555078268
33
0.0031519655603915453
34
0.0016159674851223826
35
0.0012465819017961621
36
0.0010288059711456299
37
0.0011858793441206217
38
0.0012598361354321241
39
0.0010023550130426884
40
0.003093005856499076
41
0.0013585648266598582
42
0.001349960220977664
43
0.001630208338610828
44
0.0026570868212729692
45
0.001115257851779461
46
0.0014866235433146358
47
0.002018268685787916
48
0.0038183785509318113
49
0.004640971776098013
50
0.00180583365727216
51
0.0031112059950828552
52
0.003531406633555889
53
0.0018268601270392537
54
0.002990793902426958
55
0.004001082852482796
56
0.004848792217671871
57
0.0032533113844692707
58
0.0019176919013261795
59
0.0026085232384502888
60
0.0024405091535